# Проверка модели с указанными параметрами

In [ ]:
from turbine_model import calc_turbine_power
import numpy as np

# Параметры из статьи
alpha = 0.33
gamma = 0  # градусы
beta = 0   # градусы
u = 12     # м/с
D = 98    # м

# Расчёт мощности
power = calc_turbine_power(alpha, gamma, beta, u, D)
print(f"Расчётная мощность: {power/1e6:.2f} МВт")

# Ожидаемая мощность для турбины Vestas V150-4.5 MW
P_expected = 4.5
print(f"Ожидаемая мощность: {P_expected} МВт")

# Погрешность
error = abs(power/1e6 - P_expected)/P_expected * 100
print(f"Погрешность: {error:.2f} %")

# Моделирование в FLORIS

In [1]:
# Координаты турбин в соответствии с (TableB.2:Layoutanddivisionofwindfarms) 1-я строка 13 турбин
coordinates = [[0, 0], [400, 0], [800, -800], [800, -400], [1200, 0], [1200, -800], [1600, -400], [1600, 800], [2000, 400], [2000, 0], [2000, -800], [2400, 400], [2400, -400]]

In [2]:
from floris import FlorisModel

# Инициализация модели с конфигом gch.yaml
fmodel = FlorisModel("gch.yaml")

fmodel.set(
    layout_x=[x for x, _ in coordinates],
    layout_y=[y for _, y in coordinates],
    wind_directions=[270.0],     # Направление ветра (градусы)
    wind_speeds=[8.0],           # Скорость ветра (м/с)
    turbulence_intensities=[0.1] # Турбулентность I0
)
fmodel.run()

AttributeError: The initialization for Core was given extraneous inputs: ['turbine_library']

In [ ]:
# Мощности турбин (Вт)
powers = fmodel.get_turbine_powers()  
print("Мощности:", powers / 1e6, "МВт")

# Скорости ветра на роторах
velocities = fmodel.core.flow_field.u
print("Скорости:", velocities, "м/с")

In [ ]:
import matplotlib.pyplot as plt
from floris.flow_visualization import visualize_cut_plane

# Рассчитать горизонтальную плоскость
plane = fmodel.calculate_horizontal_plane(height=90.0)

# Визуализировать с помощью специальной функции
fig, ax = plt.subplots()
visualize_cut_plane(
    plane,
    ax=ax,
    title="Поле скоростей ветра",
    color_bar_label="Скорость (м/с)",
    cmap="viridis"
)
plt.show()
